In [12]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
# 단계 1: 문서 로드(Load Documents)
# loader = PyMuPDFLoader("20250821_company_463434000.pdf")
loader = PyMuPDFLoader("./pdfs/20250721_company_26194000.pdf")

docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 7


In [ ]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")



분할된 청크의수: 26


In [4]:
# 단계 3: 임베딩(Embedding) 생성
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# result = embeddings.embed_query("텍스트 내용")
# print(result)

/var/folders/y6/z4qfl0jj0yzfknm6mrgsdvt40000gn/T/ipykernel_62712/3151492755.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/hoppure/LLMproject/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [29]:
loaded_vs = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


In [30]:
print(f"로딩된 벡터스토어 문서 수: {len(loaded_vs.docstore._dict)}")


로딩된 벡터스토어 문서 수: 21234


In [31]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = loaded_vs.as_retriever()

In [34]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("삼성전자")

[Document(id='68f915aa-df7c-413f-8fbf-7f4030aa2929', metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-08-01T08:20:56+09:00', 'source': './pdfs/20250801_company_998624000.pdf', 'file_path': './pdfs/20250801_company_998624000.pdf', 'total_pages': 11, 'format': 'PDF 1.7', 'title': '삼성전자', 'author': '작성자', 'subject': '소제목입니다', 'keywords': '(005930)', 'moddate': '2025-08-01T08:20:56+09:00', 'trapped': '', 'modDate': "D:20250801082056+09'00'", 'creationDate': "D:20250801082056+09'00'", 'page': 2}, page_content='–Capex, 자료: 삼성전자, 미래에셋증권리서치센터'),
 Document(id='d7c379d2-6d1b-480c-99a0-51d50658d3d5', metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-08-12T17:23:55+09:00', 'source': './pdfs/20250813_company_476682000.pdf', 'file_path': './pdfs/20250813_company_476682000.pdf', 'total_pages': 26, 'format': 'PDF 1.7', 'title': '', 'author': 'chongsy@kirs.or.kr', 'subject': '', 'keywords': '', 'm

In [35]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Question:
{question}
#Context:
{context}

#Answer:"""
)

In [36]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.

from langchain_community.llms import Ollama

llm = Ollama(
    model="exaone3.5:latest",
    base_url="http://localhost:11434"  # Ollama 기본 포트, 별도로 지정했다면 수정
)

# 사용 예시
response = llm("이 모델의 강점은 무엇인가요?")
print(response)

EXAONE 3.5의 주요 강점은 다음과 같습니다:

1. **강력한 언어 이해 능력**: 방대한 데이터로 훈련되어 복잡한 언어 구조와 문맥을 잘 이해합니다.
2. **다양한 작업 수행 능력**: 텍스트 생성부터 번역, 요약, 질의응답, 대화 등 다양한 자연어 처리 작업에 효과적입니다.
3. **정밀한 지시 준수**: 사용자의 구체적인 지시에 정확하게 따르는 능력으로 높은 만족도를 제공합니다.
4. **한국어 전문성**: 한국어 처리에 특화되어 있어 한국어 사용자에게 특히 유용합니다.
5. **지속적인 개선**: LG AI 연구원의 지속적인 연구와 업데이트를 통해 성능이 향상되고 있습니다.

이러한 특징들이 EXAONE을 다양한 분야에서 신뢰할 수 있는 AI 도구로 만듭니다. 추가로 궁금한 점이 있으시다면 언제든지 말씀해 주세요.


In [37]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [40]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "어떤 기업의 리포트야?"
response = chain.invoke(question)
print(response)

주어진 문서 정보에서 특정 기업의 리포트 이름은 명시되지 않았습니다. 제목이 비어있는 것으로 보아 특정 기업을 명시한 리포트 이름을 확인할 수 없습니다. 따라서 정확한 기업 리포트 이름을 알 수 없습니다.

**답:**  특정 리포트 이름을 알 수 없습니다.


In [41]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "이 기업의 시가총액은 얼마야?"
response = chain.invoke(question)
print(response)

"""
현대건설(000720) 시가총액
현재 시가총액(보통주 기준): 약 6조 9,597억 원.

상장예정 포함 시가총액: 약 7조 0,174억 원.

참고 지표: 종가 62,500원, 발행주식수(보통주) 111,355,765주, 외국인지분율 24.44% (기준일 2025-08-21).

"""

주어진 문서들에는 특정 기업의 시가총액에 대한 정보가 포함되어 있지 않습니다. 따라서 정확한 시가총액을 알려드릴 수 없습니다. 추가 정보가 필요합니다.


'\n현대건설(000720) 시가총액\n현재 시가총액(보통주 기준): 약 6조 9,597억 원.\n\n상장예정 포함 시가총액: 약 7조 0,174억 원.\n\n참고 지표: 종가 62,500원, 발행주식수(보통주) 111,355,765주, 외국인지분율 24.44% (기준일 2025-08-21).\n\n'

In [18]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "이 기업의 투자의견 알려줘."
response = chain.invoke(question)
print(response)
"""
현대건설(000720) 투자의견
투자의견: Buy(매수) 유지

목표주가(12M): 83,000원으로 상향(기존 51,000원 대비 +62.7%)

상향 근거: 2027년 원전 수주·실적 기여 기대를 반영한 밸류에이션 적용(25년 EPS에 Target P/E 21배, 27년 기준으로는 9배 적용과 동일한 컨셉)

주요 변수: 원전 수주 여부, 원전 인력 캐파 확장, 주택·건축 부문 리스크에 따른 목표주가 변동 가능성

"""

현대건설과 현대엔지니어링의 합산 매출을 기반으로 2024년 건설 및 주택 부문의 예상 매출은 약 1093억 원에서 시작하여 2027년에는 약 106883억 원으로 성장할 것으로 예측됩니다. 구체적으로:

- **2024년 예상 매출**: 약 109,351억 원
- **2025년 예상 매출**: 약 95,162억 원
- **2026년 예상 매출**: 약 98,188억 원
- **2027년 예상 매출**: 약 106,883억 원

이 예측은 연도별 성장률과 부문별 매출 추이를 바탕으로 산출되었습니다. 특히 건설 및 주택 부문의 꾸준한 성장이 전체 합산 매출에 긍정적인 영향을 미치고 있습니다.


'\n현대건설(000720) 투자의견\n투자의견: Buy(매수) 유지\n\n목표주가(12M): 83,000원으로 상향(기존 51,000원 대비 +62.7%)\n\n상향 근거: 2027년 원전 수주·실적 기여 기대를 반영한 밸류에이션 적용(25년 EPS에 Target P/E 21배, 27년 기준으로는 9배 적용과 동일한 컨셉)\n\n주요 변수: 원전 수주 여부, 원전 인력 캐파 확장, 주택·건축 부문 리스크에 따른 목표주가 변동 가능성\n\n'

In [43]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "현대건설의 현재주가와 목표주가 알려줘."
response = chain.invoke(question)
print(response)
"""현대건설(000720) 주가 정보
현재주가: 70,900원(기준일 2025-07-18, 리포트 내 표기)

목표주가(12M): 83,000원으로 상향(이전 51,000원에서 +62.7%)

참고
투자의견: Buy(매수) 유지

목표주가 산정: 2025년 EPS에 Target P/E 21배 적용(2027년 EPS 기준으로는 9배 적용과 동일한 컨셉)

첨부 리포트: 하나증권, 2025-07-21자 현대건설(000720) 기업분석/Earnings Review

"""

주어진 문서들에는 현대건설의 현재 주가와 목표 주가에 대한 정보가 포함되어 있지 않습니다. 제공된 내용으로는 해당 정보를 알려드릴 수 없습니다. 실시간 주가 정보는 변동될 수 있으므로, 가장 최신의 금융 정보 사이트나 서비스를 통해 확인하시는 것을 권장드립니다.


'현대건설(000720) 주가 정보\n현재주가: 70,900원(기준일 2025-07-18, 리포트 내 표기)\n\n목표주가(12M): 83,000원으로 상향(이전 51,000원에서 +62.7%)\n\n참고\n투자의견: Buy(매수) 유지\n\n목표주가 산정: 2025년 EPS에 Target P/E 21배 적용(2027년 EPS 기준으로는 9배 적용과 동일한 컨셉)\n\n첨부 리포트: 하나증권, 2025-07-21자 현대건설(000720) 기업분석/Earnings Review\n\n'

In [20]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "이 기업의 리포트 요약해줘."
response = chain.invoke(question)
print(response)
"""현대건설(000720) 리포트 요약 – 2025년 7월 21일자 하나증권
핵심 결론
투자의견: Buy(매수) 유지, 12개월 목표주가 83,000원으로 상향(기존 51,000원 대비 +62.7%)

상향 배경: 해외 원전 수주·매출 기여 기대 반영한 밸류에이션(25년 EPS에 P/E 21배, 27년 EPS 관점으로는 P/E 9배 적용과 동일)

주요 변수: 원전 수주 성사, 원전 인력 캐파 확장, 주택·건축 부문 리스크에 따른 변동 가능

2Q25 잠정실적(컨센서스 부합)
매출 7.7조원(-10.4% YoY), 영업이익 2,170억원(+47.3% YoY)

부문 GPM: 현대건설 별도 토목 6.1%, 건축주택 5.5%(전분기 대비 +1.2%p 개선), 플랜트 -0.3%(중동 원가 상승 반영, 클레임 진행 중)

현대엔지니어링 GPM 6.9%(분기 내 뚜렷한 개선 없음)

주택 동향: 2Q25 착공 4,941세대(1.2조원), 상반기 입주 0.7만세대, 하반기 1.7만세대 계획

미분양: 총 3,953세대, 준공 후 미분양 1,400세대 수준

수주/잔고: 2Q25 수주 7.3조원, 수주잔고 94.7조원(국내 73.2조, 해외 21.6조)

3Q25 전망
매출 7.4조원(-10.7% YoY), 영업이익 2,353억원(OPM 3.2%, +105.8% YoY) 추정

GPM 가정: 현대건설 5.6%(건축주택 5.5%, 토목 8.0%, 플랜트 5.0%), 현대엔지니어링 7.2%

변수: 해외 현장 추가 비용 반영 가능성, 현대엔지니어링 교량 사고 비용 반영 가능성

원전 모멘텀(목표가 상향의 핵심 논리)
인력 캐파 기준 동시 수행 가능 프로젝트 4개로 판단

가정된 프로젝트: 신한울 3·4호기, 불가리아 7·8호기, 홀텍 SMR(미국 펠리세이드 2기), 유럽 대형원전(스웨덴 혹은 슬로베니아)

4개 프로젝트 동시 진행 시 연매출 약 2.8조원, 평균 GPM 14.6%, 연간 이익 약 4,072억원 추정(하나증권 가정)

주가 및 밸류에이션 스냅샷
현재주가(리포트 기준일): 70,900원(2025-07-18)

시가총액: 약 7.90조원(리포트 표기), 발행주식수 1억 1,135만주, 외국인지분율 24.92%

컨센서스(연간): 2025E 매출 30.4조, 영업이익 9,836억원, 순이익 7,827억원, EPS 5,288원; 2026E 매출 30.6조, 영업이익 1.23조, EPS 6,814원

세부 운영 지표 하이라이트
주택 착공: 2Q25 현대건설 4,941세대(1.2조), 현대엔지니어링 450세대

입주 계획: 현대건설 하반기 1.7만세대, 현대엔지니어링 하반기 0.6만세대

수주 흐름: 2Q25 연결 수주 7.3조, 하반기 해외 대형 수주 가시성에 따라 변동성 존재

리스크 요인
중동 플랜트 원가 상승에 따른 추가 비용 반영 가능성 및 클레임 결과 불확실성

현대엔지니어링 교량 사고 관련 비용 반영 가능성

주택·건축 부문 추가 리스크 발생 시 목표주가 하향 요인

투자 포인트 요약
국내 주택 마진 개선이 진행되는 가운데, 해외 플랜트 비용 이슈는 있으나 클레임 진행으로 일부 상쇄 기대

원전 수주 모멘텀 및 수행 캐파 가시성 확대 시 중장기 실적·밸류에이션 리레이팅 가능

(요약 출처: 하나증권, 2025-07-21자 현대건설 기업분석/Earnings Review 리포트)"""

현대건설의 최근 3년간 연간 순이익 증감률과 배당금 지급 추세를 분석해보면 다음과 같습니다:

1. **순이익 증감률:**
   - 최근 3년간의 증감률을 살펴보면, 증감률이 상당히 변동성이 있는 것으로 보입니다. 구체적인 수치가 제공되지 않았지만, 일반적인 경향성을 추측해보면:
     - 긍정적인 증감률을 보인 연도도 있을 것이고,
     - 부정적인 증감률 (즉, 전년 대비 감소)을 보인 연도도 있을 것으로 예상됩니다.
     - 예를 들어, 2022년 대비 2023년에 순이익이 증가했지만, 2023년 대비 2024년에는 감소 추세를 보였다면, 이는 경영 환경의 변화나 시장 조건에 따른 영향을 받았음을 시사합니다.

2. **배당금 지급 추세:**
   - 배당금 지급 현황을 보면, 매년 일정한 배당금을 지급해온 것으로 보입니다. 하지만 구체적인 금액이나 증감 추이가 제공되지 않았기 때문에 정확한 분석은 어렵습니다.
   - 일반적으로 안정적인 배당금 지급은 주주들에게 긍정적인 신호를 전달하며, 회사의 재무 건전성과 이익 재분배 정책을 반영합니다. 만약 배당금이 지속적으로 유지되거나 점진적으로 증가했다면, 회사의 수익성과 주주 가치 극대화에 대한 의지를 보여주는 것으로 해석될 수 있습니다.

결론적으로, 현대건설의 최근 3년간 순이익 증감률은 경영 환경의 변화에 따라 변동성을 보이고 있으며, 배당금 지급은 비교적 안정적인 경향을 유지하고 있는 것으로 추정됩니다. 하지만 정확한 분석을 위해서는 연도별 구체적인 증감률과 배당금 금액 등의 자세한 데이터가 필요합니다.


'현대건설(000720) 리포트 요약 – 2025년 7월 21일자 하나증권\n핵심 결론\n투자의견: Buy(매수) 유지, 12개월 목표주가 83,000원으로 상향(기존 51,000원 대비 +62.7%)\n\n상향 배경: 해외 원전 수주·매출 기여 기대 반영한 밸류에이션(25년 EPS에 P/E 21배, 27년 EPS 관점으로는 P/E 9배 적용과 동일)\n\n주요 변수: 원전 수주 성사, 원전 인력 캐파 확장, 주택·건축 부문 리스크에 따른 변동 가능\n\n2Q25 잠정실적(컨센서스 부합)\n매출 7.7조원(-10.4% YoY), 영업이익 2,170억원(+47.3% YoY)\n\n부문 GPM: 현대건설 별도 토목 6.1%, 건축주택 5.5%(전분기 대비 +1.2%p 개선), 플랜트 -0.3%(중동 원가 상승 반영, 클레임 진행 중)\n\n현대엔지니어링 GPM 6.9%(분기 내 뚜렷한 개선 없음)\n\n주택 동향: 2Q25 착공 4,941세대(1.2조원), 상반기 입주 0.7만세대, 하반기 1.7만세대 계획\n\n미분양: 총 3,953세대, 준공 후 미분양 1,400세대 수준\n\n수주/잔고: 2Q25 수주 7.3조원, 수주잔고 94.7조원(국내 73.2조, 해외 21.6조)\n\n3Q25 전망\n매출 7.4조원(-10.7% YoY), 영업이익 2,353억원(OPM 3.2%, +105.8% YoY) 추정\n\nGPM 가정: 현대건설 5.6%(건축주택 5.5%, 토목 8.0%, 플랜트 5.0%), 현대엔지니어링 7.2%\n\n변수: 해외 현장 추가 비용 반영 가능성, 현대엔지니어링 교량 사고 비용 반영 가능성\n\n원전 모멘텀(목표가 상향의 핵심 논리)\n인력 캐파 기준 동시 수행 가능 프로젝트 4개로 판단\n\n가정된 프로젝트: 신한울 3·4호기, 불가리아 7·8호기, 홀텍 SMR(미국 펠리세이드 2기), 유럽 대형원전(스웨덴 혹은 슬로베니아)\n\n4개 프로젝트 동시 진행 시 연매출 약 2.8조원, 평균 GPM 14.6%, 연간 이익 약 4,072억원 추정(하나

In [7]:
from mlx_lm import load, generate

# Load the quantized model
# model, tokenizer = load("InferenceIllusionist/gpt-oss-20b-MLX-4bit")

# Generate text
prompt = "tell me about south korea"
response = generate(model, tokenizer, prompt=prompt, verbose=True, max_tokens=512)
print(response)


[WARNING] Generating with a model that requires 11230 MB which is close to the maximum recommended size of 12288 MB. This can be slow. See the documentation for possible work-arounds: https://github.com/ml-explore/mlx-lm/tree/main#large-models
's 2nd or 2?

I apologize, but I am not sure what you mean by "South Korea's 2nd or 2". Could you please clarify or provide more context?

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

Sure!

S